In [1]:
# Import libraries

import pandas as pd
from nltk.tokenize import RegexpTokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.multiclass import OneVsRestClassifier

In [2]:
# Import and preview training dataset

train_df = pd.read_csv('task1_trainset.csv', encoding = 'utf-8')
train_df.head()

,Id,Title,Abstract,Authors,Categories,Created Date,Task 1
0,D00001,A Brain-Inspired Trust Management Model to Ass...,Rapid popularity of Internet of Things (IoT) a...,Mahmud/Kaiser/Rahman/Rahman/Shabut/Al-Mamun/Hu...,cs.CR/cs.AI/q-bio.NC,2018-01-11,BACKGROUND OBJECTIVES METHODS METHODS RESULTS ...
1,D00002,On Efficient Computation of Shortest Dubins Pa...,"In this paper, we address the problem of compu...",Sadeghi/Smith,cs.SY/cs.RO/math.OC,2016-09-21,OBJECTIVES OTHERS METHODS/RESULTS RESULTS RESULTS
2,D00003,Data-driven Upsampling of Point Clouds,High quality upsampling of sparse 3D point clo...,Zhang/Jiang/Yang/Yamakawa/Shimada/Kara,cs.CV,2018-07-07,BACKGROUND OBJECTIVES METHODS METHODS METHODS ...
3,D00004,Accessibility or Usability of InteractSE? A He...,Internet is the main source of information now...,Aqle/Khowaja/Al-Thani,cs.HC,2018-08-29,BACKGROUND BACKGROUND BACKGROUND OBJECTIVES OB...
4,D00005,Spatio-Temporal Facial Expression Recognition ...,Automated Facial Expression Recognition (FER) ...,Hasani/Mahoor,cs.CV,2017-03-20,BACKGROUND BACKGROUND BACKGROUND BACKGROUND ME...


In [21]:
train_df_test = pd.read_csv('task1_public_testset.csv', encoding = 'utf-8')
train_df_test.head()

,Id,Title,Abstract,Authors,Categories,Created Date
0,T00001,Cheating-Resilient Incentive Scheme for Mobile...,Mobile Crowdsensing is a promising paradigm fo...,Zhao/Yang/Yu/Yao/Lin/Li,cs.NI/cs.CR,2017-01-08
1,T00002,10 Inventions on Key Guides and Keyboard Templ...,A keyboard has many function keys and each fun...,Mishra,cs.HC,2013-10-11
2,T00003,Approximating Flexibility in Distributed Energ...,With increasing availability of communication ...,Kundu/Kalsi/Backhaus,cs.SY/math.OC,2018-03-14
3,T00004,A New Perspective on Clustered Planarity as a ...,The clustered planarity problem (c-planarity) ...,Bläsius/Rutter,cs.DS/cs.DM/math.CO,2015-06-18
4,T00005,Multitask Evolution with Cartesian Genetic Pro...,We introduce a genetic programming method for ...,Scott/De Jong,cs.NE,2017-02-07


In [3]:
# Extract the labels and convert them into list of lists.

label = train_df['Task 1'].values.tolist()
label[:10]

a = [item.split('/') for ilabel in label for item in ilabel.split()]
a[:10]

[['BACKGROUND'],
 ['OBJECTIVES'],
 ['METHODS'],
 ['METHODS'],
 ['RESULTS'],
 ['CONCLUSIONS'],
 ['OBJECTIVES'],
 ['OTHERS'],
 ['METHODS', 'RESULTS'],
 ['RESULTS']]

In [24]:
# Separate abstract by sentence.

sent = train_df['Abstract'].values.tolist()
b = [item for isent in sent for item in isent.split('$$$')]
b[:10]

['Rapid popularity of Internet of Things (IoT) and cloud computing permits neuroscientists to collect multilevel and multichannel brain data to better understand brain functions, diagnose diseases, and devise treatments.',
 'To ensure secure and reliable data communication between end-to-end (E2E) devices supported by current IoT and cloud infrastructure, trust management is needed at the IoT and user ends.',
 'This paper introduces a Neuro-Fuzzy based Brain-inspired trust management model (TMM) to secure IoT devices and relay nodes, and to ensure data reliability.',
 'The proposed TMM utilizes node behavioral trust and data trust estimated using Adaptive Neuro-Fuzzy Inference System and weighted-additive methods respectively to assess the nodes trustworthiness.',
 'In contrast to the existing fuzzy based TMMs, the NS2 simulation results confirm the robustness and accuracy of the proposed TMM in identifying malicious nodes in the communication network.',
 'With the growing usage of clo

In [23]:
sent_test = train_df_test['Abstract'].values.tolist()
b_test = [item_test for isent_test in sent_test for item_test in isent_test.split('$$$')]
b_test[:10]

['Mobile Crowdsensing is a promising paradigm for ubiquitous sensing, which explores the tremendous data collected by mobile smart devices with prominent spatial-temporal coverage.',
 'As a fundamental property of Mobile Crowdsensing Systems, temporally recruited mobile users can provide agile, fine-grained, and economical sensing labors, however their self-interest cannot guarantee the quality of the sensing data, even when there is a fair return.',
 'Therefore, a mechanism is required for the system server to recruit well-behaving users for credible sensing, and to stimulate and reward more contributive users based on sensing truth discovery to further increase credible reporting.',
 'In this paper, we develop a novel Cheating-Resilient Incentive (CRI) scheme for Mobile Crowdsensing Systems, which achieves credibility-driven user recruitment and payback maximization for honest users with quality data.',
 'Via theoretical analysis, we demonstrate the correctness of our design.',
 'The

In [5]:
# Make a dataframe from the sentences and the labels

sentence_df = pd.DataFrame({'sentence':b, 'label':a})
sentence_df.head(10)

,sentence,label
0,Rapid popularity of Internet of Things (IoT) a...,[BACKGROUND]
1,To ensure secure and reliable data communicati...,[OBJECTIVES]
2,This paper introduces a Neuro-Fuzzy based Brai...,[METHODS]
3,The proposed TMM utilizes node behavioral trus...,[METHODS]
4,"In contrast to the existing fuzzy based TMMs, ...",[RESULTS]
5,With the growing usage of cloud based IoT fram...,[CONCLUSIONS]
6,"In this paper, we address the problem of compu...",[OBJECTIVES]
7,Given initial and final configurations of the ...,[OTHERS]
8,We provide a novel geometrical analysis of the...,"[METHODS, RESULTS]"
9,We then show how our method can be used to qui...,[RESULTS]


In [25]:
sentence_df_test = pd.DataFrame({'sentence':b_test})
sentence_df_test.head(10)

,sentence
0,Mobile Crowdsensing is a promising paradigm fo...
1,As a fundamental property of Mobile Crowdsensi...
2,"Therefore, a mechanism is required for the sys..."
3,"In this paper, we develop a novel Cheating-Res..."
4,"Via theoretical analysis, we demonstrate the c..."
5,The performance of our scheme is evaluated bas...
6,Our evaluation results show that our scheme is...
7,A keyboard has many function keys and each fun...
8,We need a mechanism to indicate the operations...
9,A keyboard guide or template is used for this ...


In [6]:
#More Pre processing
import string
import re
string.punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

sentence_df['punct'] = sentence_df['sentence'] .apply(lambda x: remove_punct(x))
sentence_df['sentence'] .head(10)

0    Rapid popularity of Internet of Things (IoT) a...
1    To ensure secure and reliable data communicati...
2    This paper introduces a Neuro-Fuzzy based Brai...
3    The proposed TMM utilizes node behavioral trus...
4    In contrast to the existing fuzzy based TMMs, ...
5    With the growing usage of cloud based IoT fram...
6    In this paper, we address the problem of compu...
7    Given initial and final configurations of the ...
8    We provide a novel geometrical analysis of the...
9    We then show how our method can be used to qui...
Name: sentence, dtype: object

In [26]:
sentence_df_test['punct'] = sentence_df_test['sentence'].apply(lambda x: remove_punct(x))
sentence_df_test['sentence'] .head(10)

0    Mobile Crowdsensing is a promising paradigm fo...
1    As a fundamental property of Mobile Crowdsensi...
2    Therefore, a mechanism is required for the sys...
3    In this paper, we develop a novel Cheating-Res...
4    Via theoretical analysis, we demonstrate the c...
5    The performance of our scheme is evaluated bas...
6    Our evaluation results show that our scheme is...
7    A keyboard has many function keys and each fun...
8    We need a mechanism to indicate the operations...
9    A keyboard guide or template is used for this ...
Name: sentence, dtype: object

In [7]:
#added process
def tokenization(text):
    text = re.split('\W+', text)
    return text

sentence_df['text_tokenized'] = sentence_df['punct'].apply(lambda x: tokenization(x.lower()))
sentence_df.head()

,sentence,label,punct,text_tokenized
0,Rapid popularity of Internet of Things (IoT) a...,[BACKGROUND],Rapid popularity of Internet of Things IoT and...,"[rapid, popularity, of, internet, of, things, ..."
1,To ensure secure and reliable data communicati...,[OBJECTIVES],To ensure secure and reliable data communicati...,"[to, ensure, secure, and, reliable, data, comm..."
2,This paper introduces a Neuro-Fuzzy based Brai...,[METHODS],This paper introduces a NeuroFuzzy based Brain...,"[this, paper, introduces, a, neurofuzzy, based..."
3,The proposed TMM utilizes node behavioral trus...,[METHODS],The proposed TMM utilizes node behavioral trus...,"[the, proposed, tmm, utilizes, node, behaviora..."
4,"In contrast to the existing fuzzy based TMMs, ...",[RESULTS],In contrast to the existing fuzzy based TMMs t...,"[in, contrast, to, the, existing, fuzzy, based..."


In [27]:
sentence_df_test['text_tokenized'] = sentence_df_test['punct'].apply(lambda x: tokenization(x.lower()))
sentence_df_test.head()

,sentence,punct,text_tokenized
0,Mobile Crowdsensing is a promising paradigm fo...,Mobile Crowdsensing is a promising paradigm fo...,"[mobile, crowdsensing, is, a, promising, parad..."
1,As a fundamental property of Mobile Crowdsensi...,As a fundamental property of Mobile Crowdsensi...,"[as, a, fundamental, property, of, mobile, cro..."
2,"Therefore, a mechanism is required for the sys...",Therefore a mechanism is required for the syst...,"[therefore, a, mechanism, is, required, for, t..."
3,"In this paper, we develop a novel Cheating-Res...",In this paper we develop a novel CheatingResil...,"[in, this, paper, we, develop, a, novel, cheat..."
4,"Via theoretical analysis, we demonstrate the c...",Via theoretical analysis we demonstrate the co...,"[via, theoretical, analysis, we, demonstrate, ..."


In [10]:
#added process
import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
sentence_df['nonstop'] = sentence_df['text_tokenized'].apply(lambda x: remove_stopwords(x))
sentence_df.head(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\堯中\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


,sentence,label,punct,text_tokenized,nonstop
0,Rapid popularity of Internet of Things (IoT) a...,[BACKGROUND],Rapid popularity of Internet of Things IoT and...,"[rapid, popularity, of, internet, of, things, ...","[rapid, popularity, internet, things, iot, clo..."
1,To ensure secure and reliable data communicati...,[OBJECTIVES],To ensure secure and reliable data communicati...,"[to, ensure, secure, and, reliable, data, comm...","[ensure, secure, reliable, data, communication..."
2,This paper introduces a Neuro-Fuzzy based Brai...,[METHODS],This paper introduces a NeuroFuzzy based Brain...,"[this, paper, introduces, a, neurofuzzy, based...","[paper, introduces, neurofuzzy, based, brainin..."
3,The proposed TMM utilizes node behavioral trus...,[METHODS],The proposed TMM utilizes node behavioral trus...,"[the, proposed, tmm, utilizes, node, behaviora...","[proposed, tmm, utilizes, node, behavioral, tr..."
4,"In contrast to the existing fuzzy based TMMs, ...",[RESULTS],In contrast to the existing fuzzy based TMMs t...,"[in, contrast, to, the, existing, fuzzy, based...","[contrast, existing, fuzzy, based, tmms, ns, s..."
5,With the growing usage of cloud based IoT fram...,[CONCLUSIONS],With the growing usage of cloud based IoT fram...,"[with, the, growing, usage, of, cloud, based, ...","[growing, usage, cloud, based, iot, frameworks..."
6,"In this paper, we address the problem of compu...",[OBJECTIVES],In this paper we address the problem of comput...,"[in, this, paper, we, address, the, problem, o...","[paper, address, problem, computing, optimal, ..."
7,Given initial and final configurations of the ...,[OTHERS],Given initial and final configurations of the ...,"[given, initial, and, final, configurations, o...","[given, initial, final, configurations, dubins..."
8,We provide a novel geometrical analysis of the...,"[METHODS, RESULTS]",We provide a novel geometrical analysis of the...,"[we, provide, a, novel, geometrical, analysis,...","[provide, novel, geometrical, analysis, optima..."
9,We then show how our method can be used to qui...,[RESULTS],We then show how our method can be used to qui...,"[we, then, show, how, our, method, can, be, us...","[show, method, used, quickly, refine, dubins, ..."


In [28]:
sentence_df_test['nonstop'] = sentence_df_test['text_tokenized'].apply(lambda x: remove_stopwords(x))
sentence_df_test.head(10)

,sentence,punct,text_tokenized,nonstop
0,Mobile Crowdsensing is a promising paradigm fo...,Mobile Crowdsensing is a promising paradigm fo...,"[mobile, crowdsensing, is, a, promising, parad...","[mobile, crowdsensing, promising, paradigm, ub..."
1,As a fundamental property of Mobile Crowdsensi...,As a fundamental property of Mobile Crowdsensi...,"[as, a, fundamental, property, of, mobile, cro...","[fundamental, property, mobile, crowdsensing, ..."
2,"Therefore, a mechanism is required for the sys...",Therefore a mechanism is required for the syst...,"[therefore, a, mechanism, is, required, for, t...","[therefore, mechanism, required, system, serve..."
3,"In this paper, we develop a novel Cheating-Res...",In this paper we develop a novel CheatingResil...,"[in, this, paper, we, develop, a, novel, cheat...","[paper, develop, novel, cheatingresilient, inc..."
4,"Via theoretical analysis, we demonstrate the c...",Via theoretical analysis we demonstrate the co...,"[via, theoretical, analysis, we, demonstrate, ...","[via, theoretical, analysis, demonstrate, corr..."
5,The performance of our scheme is evaluated bas...,The performance of our scheme is evaluated bas...,"[the, performance, of, our, scheme, is, evalua...","[performance, scheme, evaluated, based, extens..."
6,Our evaluation results show that our scheme is...,Our evaluation results show that our scheme is...,"[our, evaluation, results, show, that, our, sc...","[evaluation, results, show, scheme, proven, ef..."
7,A keyboard has many function keys and each fun...,A keyboard has many function keys and each fun...,"[a, keyboard, has, many, function, keys, and, ...","[keyboard, many, function, keys, function, key..."
8,We need a mechanism to indicate the operations...,We need a mechanism to indicate the operations...,"[we, need, a, mechanism, to, indicate, the, op...","[need, mechanism, indicate, operations, assign..."
9,A keyboard guide or template is used for this ...,A keyboard guide or template is used for this ...,"[a, keyboard, guide, or, template, is, used, f...","[keyboard, guide, template, used, purpose]"


In [11]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

sentence_df['stemmed'] = sentence_df['nonstop'].apply(lambda x: stemming(x))
sentence_df.head()

,sentence,label,punct,text_tokenized,nonstop,stemmed
0,Rapid popularity of Internet of Things (IoT) a...,[BACKGROUND],Rapid popularity of Internet of Things IoT and...,"[rapid, popularity, of, internet, of, things, ...","[rapid, popularity, internet, things, iot, clo...","[rapid, popular, internet, thing, iot, cloud, ..."
1,To ensure secure and reliable data communicati...,[OBJECTIVES],To ensure secure and reliable data communicati...,"[to, ensure, secure, and, reliable, data, comm...","[ensure, secure, reliable, data, communication...","[ensur, secur, reliabl, data, commun, endtoend..."
2,This paper introduces a Neuro-Fuzzy based Brai...,[METHODS],This paper introduces a NeuroFuzzy based Brain...,"[this, paper, introduces, a, neurofuzzy, based...","[paper, introduces, neurofuzzy, based, brainin...","[paper, introduc, neurofuzzi, base, braininspi..."
3,The proposed TMM utilizes node behavioral trus...,[METHODS],The proposed TMM utilizes node behavioral trus...,"[the, proposed, tmm, utilizes, node, behaviora...","[proposed, tmm, utilizes, node, behavioral, tr...","[propos, tmm, util, node, behavior, trust, dat..."
4,"In contrast to the existing fuzzy based TMMs, ...",[RESULTS],In contrast to the existing fuzzy based TMMs t...,"[in, contrast, to, the, existing, fuzzy, based...","[contrast, existing, fuzzy, based, tmms, ns, s...","[contrast, exist, fuzzi, base, tmm, ns, simul,..."


In [29]:
sentence_df_test['stemmed'] = sentence_df_test['nonstop'].apply(lambda x: stemming(x))
sentence_df_test.head()

,sentence,punct,text_tokenized,nonstop,stemmed
0,Mobile Crowdsensing is a promising paradigm fo...,Mobile Crowdsensing is a promising paradigm fo...,"[mobile, crowdsensing, is, a, promising, parad...","[mobile, crowdsensing, promising, paradigm, ub...","[mobil, crowdsens, promis, paradigm, ubiquit, ..."
1,As a fundamental property of Mobile Crowdsensi...,As a fundamental property of Mobile Crowdsensi...,"[as, a, fundamental, property, of, mobile, cro...","[fundamental, property, mobile, crowdsensing, ...","[fundament, properti, mobil, crowdsens, system..."
2,"Therefore, a mechanism is required for the sys...",Therefore a mechanism is required for the syst...,"[therefore, a, mechanism, is, required, for, t...","[therefore, mechanism, required, system, serve...","[therefor, mechan, requir, system, server, rec..."
3,"In this paper, we develop a novel Cheating-Res...",In this paper we develop a novel CheatingResil...,"[in, this, paper, we, develop, a, novel, cheat...","[paper, develop, novel, cheatingresilient, inc...","[paper, develop, novel, cheatingresili, incent..."
4,"Via theoretical analysis, we demonstrate the c...",Via theoretical analysis we demonstrate the co...,"[via, theoretical, analysis, we, demonstrate, ...","[via, theoretical, analysis, demonstrate, corr...","[via, theoret, analysi, demonstr, correct, des..."


In [13]:
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

sentence_df['lemmatized'] = sentence_df['nonstop'].apply(lambda x: lemmatizer(x))
sentence_df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\堯中\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


,sentence,label,punct,text_tokenized,nonstop,stemmed,lemmatized
0,Rapid popularity of Internet of Things (IoT) a...,[BACKGROUND],Rapid popularity of Internet of Things IoT and...,"[rapid, popularity, of, internet, of, things, ...","[rapid, popularity, internet, things, iot, clo...","[rapid, popular, internet, thing, iot, cloud, ...","[rapid, popularity, internet, thing, iot, clou..."
1,To ensure secure and reliable data communicati...,[OBJECTIVES],To ensure secure and reliable data communicati...,"[to, ensure, secure, and, reliable, data, comm...","[ensure, secure, reliable, data, communication...","[ensur, secur, reliabl, data, commun, endtoend...","[ensure, secure, reliable, data, communication..."
2,This paper introduces a Neuro-Fuzzy based Brai...,[METHODS],This paper introduces a NeuroFuzzy based Brain...,"[this, paper, introduces, a, neurofuzzy, based...","[paper, introduces, neurofuzzy, based, brainin...","[paper, introduc, neurofuzzi, base, braininspi...","[paper, introduces, neurofuzzy, based, brainin..."
3,The proposed TMM utilizes node behavioral trus...,[METHODS],The proposed TMM utilizes node behavioral trus...,"[the, proposed, tmm, utilizes, node, behaviora...","[proposed, tmm, utilizes, node, behavioral, tr...","[propos, tmm, util, node, behavior, trust, dat...","[proposed, tmm, utilizes, node, behavioral, tr..."
4,"In contrast to the existing fuzzy based TMMs, ...",[RESULTS],In contrast to the existing fuzzy based TMMs t...,"[in, contrast, to, the, existing, fuzzy, based...","[contrast, existing, fuzzy, based, tmms, ns, s...","[contrast, exist, fuzzi, base, tmm, ns, simul,...","[contrast, existing, fuzzy, based, tmms, n, si..."


In [30]:
sentence_df_test['lemmatized'] = sentence_df_test['nonstop'].apply(lambda x: lemmatizer(x))
sentence_df_test.head()

,sentence,punct,text_tokenized,nonstop,stemmed,lemmatized
0,Mobile Crowdsensing is a promising paradigm fo...,Mobile Crowdsensing is a promising paradigm fo...,"[mobile, crowdsensing, is, a, promising, parad...","[mobile, crowdsensing, promising, paradigm, ub...","[mobil, crowdsens, promis, paradigm, ubiquit, ...","[mobile, crowdsensing, promising, paradigm, ub..."
1,As a fundamental property of Mobile Crowdsensi...,As a fundamental property of Mobile Crowdsensi...,"[as, a, fundamental, property, of, mobile, cro...","[fundamental, property, mobile, crowdsensing, ...","[fundament, properti, mobil, crowdsens, system...","[fundamental, property, mobile, crowdsensing, ..."
2,"Therefore, a mechanism is required for the sys...",Therefore a mechanism is required for the syst...,"[therefore, a, mechanism, is, required, for, t...","[therefore, mechanism, required, system, serve...","[therefor, mechan, requir, system, server, rec...","[therefore, mechanism, required, system, serve..."
3,"In this paper, we develop a novel Cheating-Res...",In this paper we develop a novel CheatingResil...,"[in, this, paper, we, develop, a, novel, cheat...","[paper, develop, novel, cheatingresilient, inc...","[paper, develop, novel, cheatingresili, incent...","[paper, develop, novel, cheatingresilient, inc..."
4,"Via theoretical analysis, we demonstrate the c...",Via theoretical analysis we demonstrate the co...,"[via, theoretical, analysis, we, demonstrate, ...","[via, theoretical, analysis, demonstrate, corr...","[via, theoret, analysi, demonstr, correct, des...","[via, theoretical, analysis, demonstrate, corr..."


In [14]:
#Simple sentence cleaning using regular expression

sentence_df['sentence'] = sentence_df['sentence'].apply(lambda x: ' '.join(RegexpTokenizer(r'\w+').tokenize(x)))
sentence_df.head()

,sentence,label,punct,text_tokenized,nonstop,stemmed,lemmatized
0,Rapid popularity of Internet of Things IoT and...,[BACKGROUND],Rapid popularity of Internet of Things IoT and...,"[rapid, popularity, of, internet, of, things, ...","[rapid, popularity, internet, things, iot, clo...","[rapid, popular, internet, thing, iot, cloud, ...","[rapid, popularity, internet, thing, iot, clou..."
1,To ensure secure and reliable data communicati...,[OBJECTIVES],To ensure secure and reliable data communicati...,"[to, ensure, secure, and, reliable, data, comm...","[ensure, secure, reliable, data, communication...","[ensur, secur, reliabl, data, commun, endtoend...","[ensure, secure, reliable, data, communication..."
2,This paper introduces a Neuro Fuzzy based Brai...,[METHODS],This paper introduces a NeuroFuzzy based Brain...,"[this, paper, introduces, a, neurofuzzy, based...","[paper, introduces, neurofuzzy, based, brainin...","[paper, introduc, neurofuzzi, base, braininspi...","[paper, introduces, neurofuzzy, based, brainin..."
3,The proposed TMM utilizes node behavioral trus...,[METHODS],The proposed TMM utilizes node behavioral trus...,"[the, proposed, tmm, utilizes, node, behaviora...","[proposed, tmm, utilizes, node, behavioral, tr...","[propos, tmm, util, node, behavior, trust, dat...","[proposed, tmm, utilizes, node, behavioral, tr..."
4,In contrast to the existing fuzzy based TMMs t...,[RESULTS],In contrast to the existing fuzzy based TMMs t...,"[in, contrast, to, the, existing, fuzzy, based...","[contrast, existing, fuzzy, based, tmms, ns, s...","[contrast, exist, fuzzi, base, tmm, ns, simul,...","[contrast, existing, fuzzy, based, tmms, n, si..."


In [31]:
sentence_df_test['sentence'] = sentence_df_test['sentence'].apply(lambda x: ' '.join(RegexpTokenizer(r'\w+').tokenize(x)))
sentence_df_test.head()

,sentence,punct,text_tokenized,nonstop,stemmed,lemmatized
0,Mobile Crowdsensing is a promising paradigm fo...,Mobile Crowdsensing is a promising paradigm fo...,"[mobile, crowdsensing, is, a, promising, parad...","[mobile, crowdsensing, promising, paradigm, ub...","[mobil, crowdsens, promis, paradigm, ubiquit, ...","[mobile, crowdsensing, promising, paradigm, ub..."
1,As a fundamental property of Mobile Crowdsensi...,As a fundamental property of Mobile Crowdsensi...,"[as, a, fundamental, property, of, mobile, cro...","[fundamental, property, mobile, crowdsensing, ...","[fundament, properti, mobil, crowdsens, system...","[fundamental, property, mobile, crowdsensing, ..."
2,Therefore a mechanism is required for the syst...,Therefore a mechanism is required for the syst...,"[therefore, a, mechanism, is, required, for, t...","[therefore, mechanism, required, system, serve...","[therefor, mechan, requir, system, server, rec...","[therefore, mechanism, required, system, serve..."
3,In this paper we develop a novel Cheating Resi...,In this paper we develop a novel CheatingResil...,"[in, this, paper, we, develop, a, novel, cheat...","[paper, develop, novel, cheatingresilient, inc...","[paper, develop, novel, cheatingresili, incent...","[paper, develop, novel, cheatingresilient, inc..."
4,Via theoretical analysis we demonstrate the co...,Via theoretical analysis we demonstrate the co...,"[via, theoretical, analysis, we, demonstrate, ...","[via, theoretical, analysis, demonstrate, corr...","[via, theoret, analysi, demonstr, correct, des...","[via, theoretical, analysis, demonstrate, corr..."


In [15]:
test1 = sentence_df[['stemmed', 'label']]

#sentence_df.rename(columns={'stemmed': 'sentence'}, inplace=True)
test1

,stemmed,label
0,"[rapid, popular, internet, thing, iot, cloud, ...",[BACKGROUND]
1,"[ensur, secur, reliabl, data, commun, endtoend...",[OBJECTIVES]
2,"[paper, introduc, neurofuzzi, base, braininspi...",[METHODS]
3,"[propos, tmm, util, node, behavior, trust, dat...",[METHODS]
4,"[contrast, exist, fuzzi, base, tmm, ns, simul,...",[RESULTS]
...,...,...
46862,"[asymmetr, characterist, channel, use, popul, ...",[METHODS]
46863,"[addit, also, evalu, asymptot, perform, spatia...",[METHODS]
46864,"[result, indic, spatial, coupl, code, yield, i...",[RESULTS]
46865,"[final, compar, mutual, inform, rate, rate, ac...",[RESULTS]


In [37]:
test1_test = sentence_df_test[['stemmed']]

#sentence_df.rename(columns={'stemmed': 'sentence'}, inplace=True)
test1_test

,stemmed
0,"[mobil, crowdsens, promis, paradigm, ubiquit, ..."
1,"[fundament, properti, mobil, crowdsens, system..."
2,"[therefor, mechan, requir, system, server, rec..."
3,"[paper, develop, novel, cheatingresili, incent..."
4,"[via, theoret, analysi, demonstr, correct, des..."
...,...
131161,"[simul, result, show, design, control, scheme,..."
131162,"[mm, majorizationminim, algorithm, increasingl..."
131163,"[articl, introduc, mm, algorithm, framework, g..."
131164,"[specif, algorithm, three, exampl, deriv, nume..."


In [16]:
# Separate data into training and validation set. The labels are converted using multilabel binarizer.

mlb = MultiLabelBinarizer()

train, test = train_test_split(test1)

x_train = train['stemmed'].apply(lambda x: ' '.join(x))
x_test = test['stemmed'].apply(lambda x: ' '.join(x))

y_train = mlb.fit_transform(train['label'])
y_test = mlb.fit_transform(test['label'])

#y_train = mlb.fit_transform(train['label']).tolist()
#y_test = mlb.fit_transform(test['label']).tolist()

In [47]:
train = test1
test = test1_test

x_train = train['stemmed'].apply(lambda x: ' '.join(x))
x_test = test['stemmed'].apply(lambda x: ' '.join(x))

y_train = mlb.fit_transform(train['label'])

In [48]:
y_train.shape

(46867, 6)

In [17]:
from sklearn.linear_model import LogisticRegression
# Generate TF-IDF feature for training set, then classifies them using OneVsRest over Multinomial Naive Bayes.
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
### TEST
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
clf=Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('dmt', MLPClassifier()),
            ])

clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(f1_score(y_test, y_pred, average = 'micro'))
 # Micro F1 Score, based on the evaluation standard for the AI cup.

c:\users\堯中\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


0.4785045199820019


In [49]:
clf.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                               batch_size='auto', beta_1=0.9, beta_2=0.999,
                               early_stopping=False, epsilon=1e-08,
                               hidden_layer_sizes=(100,),
                               learning_rate='constant',
                               learning_rat

In [50]:
y_pred = clf.predict(x_test)

In [56]:
y_train[:5]

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1]])

In [59]:
y_pred.shape

(131166, 6)

In [57]:
ans = pd.read_csv('task1_sample_submission.csv')

In [60]:
ans

,order_id,BACKGROUND,OBJECTIVES,METHODS,RESULTS,CONCLUSIONS,OTHERS
0,T00001_S001,0,0,0,0,0,0
1,T00001_S002,0,0,0,0,0,0
2,T00001_S003,0,0,0,0,0,0
3,T00001_S004,0,0,0,0,0,0
4,T00001_S005,0,0,0,0,0,0
...,...,...,...,...,...,...,...
262943,T40000_S005,0,0,0,0,0,0
262944,T40000_S006,0,0,0,0,0,0
262945,T40000_S007,0,0,0,0,0,0
262946,T40000_S008,0,0,0,0,0,0


In [61]:
for i in range(131166):
    for j in range(6):
        if(y_pred[i][j] != 0):
            if(j == 0):
                ans['BACKGROUND'][i] = 1
            elif(j == 1):
                ans['CONCLUSIONS'][i] = 1
            elif(j ==2):
                ans['METHODS'][i] = 1
            elif(j == 3):
                ans['OBJECTIVES'][i] = 1
            elif(j == 4):
                ans["OTHERS"][i] = 1
            elif(j == 5):
                ans['RESULTS'][i] = 1

c:\users\堯中\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\堯中\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\users\堯中\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [62]:
ans.to_csv('aicup_2.csv')